In [1]:
# libraries import

import whitebox
import sys
import os
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import Markdown, display, HTML
import geopandas as gpd
from geopandas import *
from ipyleaflet import *
from sidecar import Sidecar
from ipywidgets import *
from ipywidgets.embed import embed_data
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from osgeo import gdal
from osgeo import ogr
import folium
from folium import raster_layers, plugins
from folium.plugins import MeasureControl, MousePosition
import tempfile, shutil, os
import json
import math

from shapely import geometry, ops
import time
import shutil
from shapely.wkt import loads

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return true;
}

<IPython.core.display.Javascript object>

In [3]:
def printmd(string):
    
    display(Markdown(string))

In [4]:
def change_dir():
    
    fdialog2.default_path = fdialog.selected
    fdialog2.reset()

In [5]:
def wkdchooser():
    
    global fdialog
    
    fdialog = FileChooser(
        os.getcwd(),        
        title='<b>Working directory</b> (select folder)',
        show_hidden=False,
        select_default=True,
        use_dir_icons=True,
        show_only_dirs=True
    )
    
    display(fdialog)
    
    fdialog.register_callback(change_dir)

In [6]:
def activatebut1():
    
    if fdialog2.selected[-4:] == '.tif':
        button1.disabled = False
    else:
        button1.disabled = True

In [7]:
def activatebut2():
    
    if button1.disabled == True:
        button2.disabled = True
    else:
        button2.disabled = False

In [8]:
def demchooser():
    
    global fdialog2
    
    fdialog2 = FileChooser(
        os.getcwd(),       
        title='<b>Input preprocessed dem</b> (select *.tif file)',
        show_hidden=False,
        select_default=True,
        use_dir_icons=True,
        show_only_dirs=False
    )   
    
    display(fdialog2) 
    
    fdialog2.register_callback(activatebut1)

In [9]:
def streamthresh():
    
    global txt1
    
    streamthresh_def1 = 2000
    
    printmd('**Stream length threshold for basin generation:**')
    txt1 = widgets.Text(value=str(streamthresh_def1))
    
    display(txt1)

In [10]:
def basin_gen():
    
    global check0
    
    check0 = widgets.Checkbox(
        value=False,
        description='Only basin generation (to confirm distribution)',
        disabled=False,
        indent=False
    )
    
    display(check0)

In [11]:
def preliminary():
    
    global check1
    
    check1 = widgets.Checkbox(
        value=False,
        description='Delete preliminary data',
        disabled=False,
        indent=False
    )
    
    display(check1)

In [12]:
def raster():
      
    transform_raster()
    HillShade()
    
    #Open raster file
    driver=gdal.GetDriverByName('GTiff')
    driver.Register() 
    ds = gdal.Open(wdir+'DEM_HS.tif') 
    if ds is None:
        print('Could not open')      
   
    #Get coordinates, cols and rows
    geotransform = ds.GetGeoTransform()
    cols = ds.RasterXSize 
    rows = ds.RasterYSize 
    
    #Get extent
    xmin=geotransform[0]
    ymax=geotransform[3]
    xmax=xmin+cols*geotransform[1]
    ymin=ymax+rows*geotransform[5]
    
    #Raster convert to array in numpy
    bands = ds.RasterCount
    band=ds.GetRasterBand(1)
    dataset= band.ReadAsArray(0,0,cols,rows)
    dataimage=dataset
    dataimage[dataimage[:,:]==-340282346638528859811704183484516925440.000]=0 
    
    with out3:
        display("raster visualization, wait please..")

    #Visualization in folium       
    raster_layers.ImageOverlay(
        image=dataimage,
        name='DEM (hillshade)',
        bounds=[[ymin, xmin], [ymax, xmax]],
        colormap=lambda x: (0, 0, 1, x) #plasma, viridis, inferno, magma, cividis        
    ).add_to(map1)
    
    print("raster_layer done")

In [13]:
def joinValley(fid, p1, p2, step):
    
    global inter_valley_shp
   
    #searching the lower valley
    sel21 = inter_valley_shp.loc[inter_valley_shp['point1'] == p2]   
    
    if sel21.empty:                     
        return ()       
    
    else:
        fid21 = sel21['FID'].iloc[0]        
        current = inter_valley_shp.loc[inter_valley_shp['FID'] == fid]        
        m_valley_sel = sel21['mValley'].iloc[0]       
        
        if m_valley_sel != 'yes':            
            #azimuth range - to join another valley
            st_a = int(current['aziDN']-50.0)
            end_a = int(current['aziDN']+50.0)
            sel_a = int(sel21['aziUP']) 
            
            if st_a < 0: 
                st_a = st_a + 360
                podminka = (sel_a in range(st_a, 360)) or (sel_a in range(0, end_a))
            elif end_a > 360:
                end_a = end_a - 360
                podminka = (sel_a in range(st_a, 360)) or (sel_a in range(0, end_a))
            else:            
                podminka = sel_a in range(st_a, end_a)            

            if podminka:
                geom_current =current.geometry.iloc[0]
                geom_sel = sel21.geometry.iloc[0]

                multi_line = geometry.MultiLineString([geom_current, geom_sel])                
                merged_line = ops.linemerge(multi_line)

                gs_ml = GeoSeries(merged_line)
                inter_valley_shp.loc[inter_valley_shp['FID'] == fid, 'geometry'] = gs_ml.values                
                inter_valley_shp.loc[inter_valley_shp['FID'] == fid, 'DN_ELEV'] = float(sel21['DN_ELEV'])
                inter_valley_shp.loc[inter_valley_shp['FID'] == fid, 'LENGTH'] = float(current['LENGTH']) + float(sel21['LENGTH'])
                inter_valley_shp.loc[inter_valley_shp['FID'] == fid, 'AVG_SLOPE'] = -999    

                p2new = sel21['point2'].iloc[0]  
                inter_valley_shp.loc[inter_valley_shp['FID'] == fid, 'point2'] = p2new             

                inter_valley_shp.loc[inter_valley_shp['FID'] == fid, 'pointMid'] = -999                
                inter_valley_shp.loc[inter_valley_shp['FID'] == fid, 'aziDN'] = float(sel21['aziDN'])

                inter_valley_shp.loc[inter_valley_shp['FID'] == fid21, 'delet'] = "yes"
               
                #one more time
                step = step + 1
                joinValley(fid, p1, p2new, step)
            else:                  
                if step == 1:
                    additionalSourceZone(fid, p1)                
                return ()
        else:           
            return ()
        
    

In [14]:
def additionalSourceZone(fid, p1):   
    
    global inter_valley_shp
    
    global sourceBasin_list
    global sourceValley_list 
    
    global basins_shp
   
    #removing incorrectly added source zones
    sourceBasin_list = sourceBasin_list[:-1]
    sourceValley_list = sourceValley_list[:-1]    
    
    bas11 = sel11['BASIN'].iloc[0]   
    fid_bas = sel11['FID'].iloc[0] 
    basins_shp.loc[basins_shp['VALUE'] == bas11, 'blind'] = 'yes'
    inter_valley_shp.loc[inter_valley_shp['point1'] == p1, 'blind'] = 'yes'

    bas_sel = basins_shp.loc[basins_shp['VALUE'] == bas11]
    inter_sel = inter_valley_shp.loc[inter_valley_shp['point1'] == p1]
    
    #defining of new source zone

    bas_sel_shp = tempPath + '/bas_sel.shp'
    bas_sel.to_file(bas_sel_shp)               

    inter_sel_shp = tempPath + '/inter_sel.shp'
    inter_sel.to_file(inter_sel_shp)                

    demOne_tif = tempPath + '/demOne.tif'                
    #DEM clipping by one basin
    wbt.clip_raster_to_polygon(
        dem, 
        bas_sel_shp, 
        demOne_tif
    )

    quantileOne_tif = tempPath + '/quantileOne.tif'
    #elevation splitting to 5 quartiles
    wbt.quantiles(
        demOne_tif, 
        quantileOne_tif, 
        num_quantiles=4
    )

    quantiles_shp = tempPath + '/quantiles.shp'
    #quartile raster to polygon
    wbt.raster_to_vector_polygons(
        quantileOne_tif, 
        quantiles_shp
    )

    #quartile selection
    quantiles2 = gpd.read_file(quantiles_shp)
    quant = quantiles2.loc[quantiles2['VALUE'] == 1.0]
    quant_shp = tempPath + '/quant.shp'
    quant.to_file(quant_shp)  

    quant_lines_shp = tempPath + '/quant_lines.shp'
    #quartile polygon to line
    wbt.polygons_to_lines(
        quant_shp, 
        quant_lines_shp
    )

    pour_shp = tempPath + '/pour_point.shp'
    #pour point definition by intersection of qurtile (line) and talweg line
    wbt.line_intersections(
        inter_sel_shp, 
        quant_lines_shp, 
        pour_shp
    )

    #upper point selection
    points = gpd.read_file(pour_shp)
    point = points.loc[points['FID'] == 1]
    pour_shp2 = tempPath + '/pour.shp'
    point.to_file(pour_shp2) 
    
    
    interOne_tiff = tempPath + '/interOne.tif'
    #talweg line to  raster
    wbt.vector_lines_to_raster(
        inter_sel_shp, 
        interOne_tiff, 
        field="FID", 
        nodata=True, 
        cell_size=None, 
        base=flow_acc     
    )
    
    pour_shp3 = tempPath + '/pour_point3.shp'
    #pour point moves to talweg
    wbt.jenson_snap_pour_points(
        pour_shp2, 
        interOne_tiff, 
        pour_shp3, 
        30.0
    )
    
    d8_one_tif = tempPath +'/d8_one.tif'
    #d8 pointer raster clip
    wbt.clip_raster_to_polygon(
        d8_pointer, 
        bas_sel_shp, 
        d8_one_tif
    )
    
    watershed_tif = tempPath +'/watershed.tif'
    #new watershed computation .. source zone
    wbt.watershed(
        d8_one_tif, 
        pour_shp3, 
        watershed_tif
    )

    sourceZone_shp = tempPath + '/sourceZone.shp'
    #watershad raster to polygon ... source zone
    wbt.raster_to_vector_polygons(
        watershed_tif, 
        sourceZone_shp
    )
    sourceZone = gpd.read_file(sourceZone_shp)   
    
    # talweg of new source zone
    sourceValley = gpd.clip(inter_sel, sourceZone)    

    #adding of source zone and talweg to shps
    sourceZone.loc[sourceZone['FID'] == 1, 'FID'] = fid_bas
    sourceZone.loc[sourceZone['VALUE'] == 1.0, 'VALUE'] = bas11
    sourceBasin_list = pd.concat([sourceBasin_list, sourceZone])
    sourceValley_list = pd.concat([sourceValley_list, sourceValley])  
    

In [15]:
def calculation():  
    
    global bas_path    

    global inter_valley_shp
    
    global sourceBasin_list
    global sourceValley_list
    
    global basins_shp
    global sel11
    
    global tempPath
    
    global flow_acc
    global d8_pointer
      
    flow_t = 3000    
    
    # D8 pointer (flow pointer grid)
    d8_pointer = wdir +'flow_pointer_grid.tif'
    wbt.d8_pointer(
        dem, 
        d8_pointer, 
        esri_pntr=False
    )  
    
    with out3:
        display("flow pointer grid - done")

    # flow accumulation
    flow_acc = wdir + 'flow_accum.tif'
    wbt.d8_flow_accumulation(
        d8_pointer, 
        flow_acc, 
        out_type="cells", 
        log=False, 
        clip=False, 
        pntr=True, 
        esri_pntr=False
    )
    
    with out3:
        display("flow accumulation - done")
    
    # STREAMS
    streams = wdir +'streams.tif'
    wbt.extract_streams(
        flow_acc, 
        streams, 
        flow_t,
        zero_background=False
    )
    #remove short streams
    long_streams = wdir +'long_streams.tif'
    wbt.remove_short_streams(
        d8_pointer, 
        streams, 
        long_streams, 
        int(stream_t), 
        esri_pntr=False
    )
    
    with out3:
        display("streams - done")
    
    # subbasins
    subbasins = wdir +'basins' + stream_t + '.tif'
    wbt.subbasins(
        d8_pointer, 
        long_streams, 
        subbasins, 
        esri_pntr=False
    )
   
    # subbasins to polygons
    sub_pol =wdir +'basins_pol' + stream_t + '.shp'
    wbt.raster_to_vector_polygons(subbasins, sub_pol)
    
    with out3:
        display("basins - done")
    
    
    if bGen_only == False:
        #--------------------------------------------------    
        #--------------------------------------------------    
        # longest flowpath
        inter_valley = wdir + 'intersect_valley.shp'
        wbt.longest_flowpath(
            dem, 
            subbasins,
            inter_valley
        )   

        with out3:
            display("intersect_valley - done")

        #-------------------------------------------------------------------------------------    
        #-------------------------------------------------------------------------------------   

        inter_valley_shp = gpd.read_file(inter_valley)
        inter_valley_shp = inter_valley_shp.assign(point1="")
        inter_valley_shp = inter_valley_shp.assign(pointMid="")
        inter_valley_shp = inter_valley_shp.assign(point2="")
        inter_valley_shp = inter_valley_shp.assign(aziUP=-999)
        inter_valley_shp = inter_valley_shp.assign(aziDN=-999)
        inter_valley_shp = inter_valley_shp.assign(slope=-999)
        inter_valley_shp = inter_valley_shp.assign(eleDif=-999)

        basins_shp = gpd.read_file(sub_pol)
        basins_shp = basins_shp.assign(blind="no")  

        demRaster = rasterio.open(dem)

        for i in np.arange(0,len(inter_valley_shp)):       
            geom = inter_valley_shp['geometry'][i] 

            n_geom = len(geom.coords)
            mid_i = int(n_geom/2)

            #first and last point of the line
            p1 = str(geom.coords[0])
            pMid = str(geom.coords[mid_i])
            p2 = str(geom.coords[-1]) 
            inter_valley_shp.loc[i, 'point1'] = p1
            inter_valley_shp.loc[i, 'pointMid'] = pMid
            inter_valley_shp.loc[i, 'point2'] = p2    

            #azimuth of line        
            x1 = float(p1[1:-1].split(", ")[0])
            y1 = float(p1[1:-1].split(", ")[1])
            xM = float(pMid[1:-1].split(", ")[0])
            yM = float(pMid[1:-1].split(", ")[1])
            x2 = float(p2[1:-1].split(", ")[0])
            y2 = float(p2[1:-1].split(", ")[1])        

            #up_elev
            row1, col1 = demRaster.index(x1,y1)    
            up_ele = demRaster.read(1)[row1,col1]
            inter_valley_shp.loc[i, 'UP_ELEV'] = up_ele 
            #dn_elev
            row2, col2 = demRaster.index(x2,y2)    
            dn_ele = demRaster.read(1)[row2,col2] 
            inter_valley_shp.loc[i, 'DN_ELEV'] = dn_ele 
            #elevation difference
            ele_dif = up_ele - dn_ele
            inter_valley_shp.loc[i, 'eleDif'] = ele_dif 

            #azimuth 2x
            azi_up = math.degrees(math.atan2((x1-xM),(y1-yM)))
            azi_up = azi_up + 180
            if azi_up == 360.00:
                azi_up = 0.0            
            inter_valley_shp.loc[i, 'aziUP'] = azi_up 

            azi_dn = math.degrees(math.atan2((xM-x2),(yM-y2)))
            azi_dn = azi_dn + 180
            if azi_dn == 360.00:
                azi_dn = 0.0            
            inter_valley_shp.loc[i, 'aziDN'] = azi_dn 

            #slope
            dist= math.dist([x1, y1], [x2, y2])
            d_ele = up_ele - dn_ele
            slope = math.degrees(math.asin(d_ele/dist))
            inter_valley_shp.loc[i, 'slope'] = slope 

        with out3:
            display("talweg parameters calculation - done")


        # BORDER VALLEYS REMOVAL -----------------------------------------

        #DEM mask
        oneRaster = wdir + "oneRaster.tif"
        wbt.greater_than(
            dem, 
            0.0, 
            oneRaster
        )

        #DEM mask to polygon
        one_shp = wdir + 'extent.shp'
        wbt.raster_to_vector_polygons(
            oneRaster, 
            one_shp
        )

        #boundary polygon to line
        boundary_shp = wdir + 'boundary.shp'    
        wbt.polygons_to_lines(
            one_shp, 
            boundary_shp
        )

        boundary = gpd.read_file(boundary_shp)
        boundary_geom = boundary.iloc[0].geometry
        basins_shp = basins_shp.assign(out="no")
        inter_valley_shp = inter_valley_shp.assign(out="no")

        for i in np.arange(0,len(basins_shp)): 

            povodi = basins_shp.iloc[i]
            povodi_geom = povodi.geometry

            if boundary_geom.touches(povodi_geom) == True:
                
                povodiValue = povodi.VALUE
                basins_shp.loc[basins_shp["VALUE"]==povodiValue,"out"] = 'yes'          
                inter_valley_shp.loc[inter_valley_shp["BASIN"] == povodiValue, 'out'] = 'yes'

        basins_shp = basins_shp.loc[basins_shp['out'] != 'yes']
        inter_valley_shp = inter_valley_shp.loc[inter_valley_shp['out'] != 'yes']

        with out3:
            display("border valley detection - done")


        # BORDER VALLEYS REMOVAL -----------------------------------------  
        # -------------------------------------------------------------------


        # IDENTIFICATION OF MAIN VALLEYS --------------------------------------
        inter_valley_shp = inter_valley_shp.assign(mValley="no")
        selMainValleys = inter_valley_shp.loc[inter_valley_shp['slope'] < 5].loc[inter_valley_shp['eleDif'] < 50]

        while len(selMainValleys)>0:

            fid = selMainValleys.iloc[0]['FID']
            p2 = selMainValleys.iloc[0]['point2'] #lower point of valley

            #marking as the main valley
            selMainValleys.loc[selMainValleys['FID'] == fid, 'mValley'] = 'yes'
            inter_valley_shp.loc[inter_valley_shp['FID'] == fid, 'mValley'] = 'yes'

            #searching the lower valley
            sel21_fid = inter_valley_shp.loc[inter_valley_shp['point1'] == p2]['FID']
            while len(sel21_fid)>0:      
                sel21_fid = int(sel21_fid)

                mVal = inter_valley_shp.loc[inter_valley_shp['FID'] == sel21_fid, 'mValley']
                mVal = mVal.iloc[0]

                if mVal !='yes':

                    selMainValleys.loc[selMainValleys['FID'] == sel21_fid, 'mValley'] = 'yes'
                    inter_valley_shp.loc[inter_valley_shp['FID'] == sel21_fid, 'mValley'] = 'yes'

                    #is there following valley?
                    p2 = inter_valley_shp.loc[inter_valley_shp['FID'] == sel21_fid]['point2']
                    p2 = p2.iloc[0]

                    sel21_fid = inter_valley_shp.loc[inter_valley_shp['point1'] == p2]['FID']
                else:
                    break

            #exclusion of already passed main valleys from the next cycle
            selMainValleys = selMainValleys.loc[selMainValleys['mValley'] != 'yes']      

        with out3:
            display("main valleys - done")

        # ---------------------------------------------------------------------------------------------------------
        # ---------------------------------------------------------------------------------------------------------
        # blind valleys

        # going through valleys up to down - joining (not main valleys)
        inter_valley_shp = inter_valley_shp.assign(delet="no")   
        inter_valley_shp = inter_valley_shp.assign(blind="no") 
        smallValleys = inter_valley_shp.loc[inter_valley_shp['mValley'] != 'yes']

        #temporary folder
        tempPath = wdir + "temp"
        tempExist = os.path.exists(tempPath)
        if not tempExist:
            os.makedirs(tempPath)

        with out3:
                label_i = Label()
                display(label_i)

        sourceBasin_list = gpd.GeoDataFrame()
        sourceValley_list = gpd.GeoDataFrame()

        for i in np.arange(0,len(smallValleys)):

            label_i.value = str(i) + " / " + str(len(smallValleys)-1)

            fid = smallValleys.iloc[i]['FID']
            p1 = smallValleys.iloc[i]['point1']
            p2 = smallValleys.iloc[i]['point2']
            delete = smallValleys.iloc[i]['delet']      

            # is it blind valley?
            sel12 = smallValleys.loc[smallValleys['point2'] == p1]

            # valley is blind.. can I join the lower valley?
            if sel12.empty and delete != 'yes':

                #selected talweg
                sel11 = inter_valley_shp.loc[inter_valley_shp['point1'] == p1]            

                if sel11.empty:
                    with out4:
                        display('neni shoda - problem?') #that should not happen
                else:                
                    #middle valley, followed another lower one?                
                    sel21 = smallValleys.loc[smallValleys['point1'] == p2]

                    if sel21.empty:
                        #not part of large valley ---------------------------------------   

                        bas11 = sel11['BASIN'].iloc[0]  
                        fid_bas = sel11['FID'].iloc[0] 
                        basins_shp.loc[basins_shp['VALUE'] == bas11, 'blind'] = 'yes'
                        inter_valley_shp.loc[inter_valley_shp['point1'] == p1, 'blind'] = 'yes'

                        bas_sel = basins_shp.loc[basins_shp['VALUE'] == bas11]
                        inter_sel = inter_valley_shp.loc[inter_valley_shp['point1'] == p1]

                        #source zone definition
                        
                        bas_sel_shp = tempPath + '/bas_sel.shp'
                        bas_sel.to_file(bas_sel_shp)               

                        inter_sel_shp = tempPath + '/inter_sel.shp'
                        inter_sel.to_file(inter_sel_shp)                

                        demOne_tif = tempPath + '/demOne.tif'                
                        #DEM clipping by one basin
                        wbt.clip_raster_to_polygon(
                            dem, 
                            bas_sel_shp, 
                            demOne_tif
                        )

                        quantileOne_tif = tempPath + '/quantileOne.tif'
                        #elevation splitting to 5 quartiles
                        wbt.quantiles(
                            demOne_tif, 
                            quantileOne_tif, 
                            num_quantiles=5
                        )

                        quantiles_shp = tempPath + '/quantiles.shp'
                        #quartile raster to polygon
                        wbt.raster_to_vector_polygons(
                            quantileOne_tif, 
                            quantiles_shp
                        )

                        #quartile selection
                        quantiles2 = gpd.read_file(quantiles_shp)
                        quant = quantiles2.loc[quantiles2['VALUE'] == 1.0]
                        quant_shp = tempPath + '/quant.shp'
                        quant.to_file(quant_shp)  

                        quant_lines_shp = tempPath + '/quant_lines.shp'
                        #quartile polygon to line
                        wbt.polygons_to_lines(
                            quant_shp, 
                            quant_lines_shp
                        )

                        pour_shp = tempPath + '/pour_point.shp'
                        #pour point definition by intersection of quartile (line) and talweg line
                        wbt.line_intersections(
                            inter_sel_shp, 
                            quant_lines_shp, 
                            pour_shp
                        )

                        #upper point selection
                        points = gpd.read_file(pour_shp)
                        point = points.loc[points['FID'] == 1]
                        pour_shp2 = tempPath + '/pour.shp'
                        point.to_file(pour_shp2) 

                        interOne_tiff = tempPath + '/interOne.tif'
                        #talweg line to raster
                        wbt.vector_lines_to_raster(
                            inter_sel_shp, 
                            interOne_tiff, 
                            field="FID", 
                            nodata=True, 
                            cell_size=None, 
                            base=flow_acc     
                        )

                        pour_shp3 = tempPath + '/pour_point3.shp'
                        #pour point moves to talweg
                        wbt.jenson_snap_pour_points(
                            pour_shp2, 
                            interOne_tiff, 
                            pour_shp3, 
                            30.0
                        )

                        d8_one_tif = tempPath +'/d8_one.tif'
                        #d8 pointer raster clip
                        wbt.clip_raster_to_polygon(
                            d8_pointer, 
                            bas_sel_shp, 
                            d8_one_tif
                        )

                        watershed_tif = tempPath +'/watershed.tif'
                        #new watershed computation .. source zone
                        wbt.watershed(
                            d8_one_tif, 
                            pour_shp3, 
                            watershed_tif
                        )

                        sourceZone_shp = tempPath + '/sourceZone.shp'
                        #watershed raster to polygon ... source zone
                        wbt.raster_to_vector_polygons(
                            watershed_tif, 
                            sourceZone_shp
                        )
                        sourceZone = gpd.read_file(sourceZone_shp)   

                        # talweg of new source zone
                        sourceValley = gpd.clip(inter_sel, sourceZone)

                        #adding of source zone and rtalweg to shps
                        sourceZone.loc[sourceZone['FID'] == 1, 'FID'] = fid_bas
                        sourceZone.loc[sourceZone['VALUE'] == 1.0, 'VALUE'] = bas11
                        sourceBasin_list = pd.concat([sourceBasin_list, sourceZone])
                        sourceValley_list = pd.concat([sourceValley_list, sourceValley])         

                    else:
                        # part of large valley -----------------------------          

                        bas11 = sel11['BASIN'].iloc[0]
                        bas_sel2 = basins_shp.loc[basins_shp['VALUE'] == bas11]
                        inter_sel2 = inter_valley_shp.loc[inter_valley_shp['FID'] == fid]

                        sourceBasin_list = pd.concat([sourceBasin_list, bas_sel2])
                        sourceValley_list = pd.concat([sourceValley_list, inter_sel2])  

                        step = 1
                        joinValley(fid, p1, p2, step) 
                        
                    with out2:
                        out2.clear_output()      

        
        sourceValley_list = sourceValley_list.assign(azi=-999)
        sourceValley_list = sourceValley_list.assign(azi_st=-999)
        
        #recalculation of source valleys parameters
        for i in np.arange(0,len(sourceValley_list)):

            sv_geom = sourceValley_list.iloc[i]['geometry']    
            sv_fid = sourceValley_list.iloc[i]['FID']     

            if sv_geom.geom_type == 'MultiLineString':
                sv_wkt = sv_geom.wkt
                #multistring to linestring
                line_wkt = sv_wkt[5:].replace("((", "(").replace("),",",").replace(", (",", ").replace("))",")")
                line_geom = loads(line_wkt)
                gs_line = GeoSeries(line_geom)

                sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'geometry'] = gs_line.values
                sv_geom = sourceValley_list.iloc[i]['geometry']

            #new length
            sv_len = sv_geom.length
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'LENGTH'] = sv_len

            n_geom = len(sv_geom.coords)
            mid_i = int(n_geom/2)

            #first and last point of the line
            p1 = str(sv_geom.coords[0])
            pMid = str(sv_geom.coords[mid_i])
            p2 = str(sv_geom.coords[-1])     

            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'point1'] = p1
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'pointMid'] = pMid
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'point2'] = p2    

            #azimuth of line        
            x1 = float(p1[1:-1].split(", ")[0])
            y1 = float(p1[1:-1].split(", ")[1])
            xM = float(pMid[1:-1].split(", ")[0])
            yM = float(pMid[1:-1].split(", ")[1])
            x2 = float(p2[1:-1].split(", ")[0])
            y2 = float(p2[1:-1].split(", ")[1])   

            #up_elev
            row1, col1 = demRaster.index(x1,y1)    
            up_ele = demRaster.read(1)[row1,col1]
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'UP_ELEV'] = up_ele 
            #dn_elev
            row2, col2 = demRaster.index(x2,y2)    
            dn_ele = demRaster.read(1)[row2,col2] 
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'DN_ELEV'] = dn_ele 
            #elevation difference
            ele_dif = up_ele - dn_ele
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'eleDif'] = ele_dif 

            #azimuth 2x
            azi_up = math.degrees(math.atan2((x1-xM),(y1-yM)))
            azi_up = azi_up + 180
            if azi_up == 360.00:
                azi_up = 0.0            
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'aziUP'] = azi_up 

            azi_dn = math.degrees(math.atan2((xM-x2),(yM-y2)))
            azi_dn = azi_dn + 180
            if azi_dn == 360.00:
                azi_dn = 0.0            
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'aziDN'] = azi_dn 
            
            #source zone talweg azimuth
            azi = math.degrees(math.atan2((x2-x1),(y2-y1)))               
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'azi'] = azi 
            if azi < 0.0:
                azi_st = azi * (-1.0)
            else:
                azi_st = azi
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'azi_st'] = azi_st 
            
            #slope
            dist= math.dist([x1, y1], [x2, y2])
            d_ele = up_ele - dn_ele
            slope = math.degrees(math.asin(d_ele/dist))
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'slope'] = slope  
            sourceValley_list.loc[sourceValley_list['FID'] == sv_fid, 'AVG_SLOPE'] = slope

        with out3:
            display("recalculation of source valleys parameters - done")

        
        #removal of redudant columns
        del sourceBasin_list['out'];del sourceBasin_list['blind']
        #shp export
        sourceBasin_shp = wdir+ 'sourceBasins.shp'
        sourceBasin_list.to_file(sourceBasin_shp)
        
        #removal of redudant columns
        del sourceValley_list['point1']; del sourceValley_list['pointMid']; del sourceValley_list['point2']; del sourceValley_list['aziUP']; del sourceValley_list['aziDN']; del sourceValley_list['out']; del sourceValley_list['delet']; del sourceValley_list['mValley']
        #shp export        
        sourceValley_shp = wdir+ 'sourceValleys.shp'
        sourceValley_list.to_file(sourceValley_shp)

        # deleting a temporary folder
        if os.path.isdir(tempPath):
            shutil.rmtree(tempPath)

        final_inter = inter_valley_shp.loc[inter_valley_shp['delet'] != 'yes']
        #parameters recalculation of other valleys
        for i in np.arange(0,len(final_inter)): 

            fi_geom = final_inter.iloc[i]['geometry']    
            fi_fid = final_inter.iloc[i]['FID'] 

            #new length
            fi_len = fi_geom.length
            final_inter.loc[final_inter['FID'] == fi_fid, 'LENGTH'] = fi_len

            n_geom = len(fi_geom.coords)
            mid_i = int(n_geom/2)

            #first and last point of the line
            p1 = str(fi_geom.coords[0])
            pMid = str(fi_geom.coords[mid_i])
            p2 = str(fi_geom.coords[-1])
            final_inter.loc[final_inter['FID'] == fi_fid, 'point1'] = p1
            final_inter.loc[final_inter['FID'] == fi_fid, 'pointMid'] = pMid
            final_inter.loc[final_inter['FID'] == fi_fid, 'point2'] = p2    

            #azimuth of line        
            x1 = float(p1[1:-1].split(", ")[0])
            y1 = float(p1[1:-1].split(", ")[1])
            xM = float(pMid[1:-1].split(", ")[0])
            yM = float(pMid[1:-1].split(", ")[1])
            x2 = float(p2[1:-1].split(", ")[0])
            y2 = float(p2[1:-1].split(", ")[1])        

            #up_elev
            row1, col1 = demRaster.index(x1,y1)    
            up_ele = demRaster.read(1)[row1,col1]
            final_inter.loc[final_inter['FID'] == fi_fid, 'UP_ELEV'] = up_ele 
            #dn_elev
            row2, col2 = demRaster.index(x2,y2)    
            dn_ele = demRaster.read(1)[row2,col2] 
            final_inter.loc[final_inter['FID'] == fi_fid, 'DN_ELEV'] = dn_ele 
            #elevation difference
            ele_dif = up_ele - dn_ele
            final_inter.loc[final_inter['FID'] == fi_fid, 'eleDif'] = ele_dif 

            #azimuth 2x
            azi_up = math.degrees(math.atan2((x1-xM),(y1-yM)))
            azi_up = azi_up + 180
            if azi_up == 360.00:
                azi_up = 0.0            
            final_inter.loc[final_inter['FID'] == fi_fid, 'aziUP'] = azi_up 

            azi_dn = math.degrees(math.atan2((xM-x2),(yM-y2)))
            azi_dn = azi_dn + 180
            if azi_dn == 360.00:
                azi_dn = 0.0            
            final_inter.loc[final_inter['FID'] == fi_fid, 'aziDN'] = azi_dn 

            #slope
            dist= math.dist([x1, y1], [x2, y2])
            d_ele = up_ele - dn_ele
            slope = math.degrees(math.asin(d_ele/dist))
            final_inter.loc[final_inter['FID'] == fi_fid, 'slope'] = slope   
            final_inter.loc[final_inter['FID'] == fi_fid, 'AVG_SLOPE'] = slope

        with out3:
            display("valley parameter recalculation - done")
            
        with out2:
                out2.clear_output()

                
        #removal of redudant columns
        del final_inter['point1'];  del final_inter['pointMid'];  del final_inter['point2'];  del final_inter['aziUP'];  del final_inter['aziDN'];  del final_inter['out'];  del final_inter['delet']
        #shp export
        inter_valley2 = wdir + 'intersect_valley2.shp'
        final_inter.to_file(inter_valley2) 

        #removal of redudant columns
        del basins_shp['out']
        #shp export
        basins2 = wdir + 'basins2.shp'
        basins_shp.to_file(basins2)         
        
   
    #display on map
    if bGen_only == True:
        bas_path = sub_pol
    else: 
        style_function2 = lambda x: {"weight":2, 
                                     'color':'#4292c6',                          
                                     'fillOpacity': 1}
        iv_shp =  wdir + 'intersect_valley2.shp'
        iv_data = gpd.read_file(iv_shp)   
        iv_data = iv_data.to_crs("EPSG:32638")
        folium.GeoJson(iv_data,
                       style_function=style_function2, 
                        name='inter valley',
                        control=True        
                        ).add_to(map1)
        
        bas_path = basins2
    bas_shp = gpd.read_file(bas_path)   
    bas_shp = bas_shp.to_crs("EPSG:32638")
    bas_shp.to_file(bas_path)       
    
    #add basins shp to map
    basins_gdf = gpd.read_file(bas_path).to_crs("EPSG:4326")    
    style = {'fillColor': 'none', 
              'color': '#FF1908', 
              'weight' : 1}    
    folium.GeoJson(data=basins_gdf["geometry"],
                   name='basins_' + stream_t,
                   style_function=lambda x:style
                   ).add_to(map1)
    
    #add DEM to map    
    raster()  
    #display map
    display_map()    
    
    driver = ogr.GetDriverByName("ESRI Shapefile")    
    #delete preliminary data
    if delete_data == True:
        os.remove(subbasins)
        os.remove(flow_acc)
        os.remove(d8_pointer)
        os.remove(streams)
        os.remove(long_streams)        
        os.remove(output_raster)
        os.remove(output_raster_HS)
    if bGen_only == False:        
        driver.DeleteDataSource(sub_pol)
        driver.DeleteDataSource(inter_valley)
        os.remove(oneRaster)
        driver.DeleteDataSource(boundary_shp)          
    
    with out3:
        printmd('**PROCESSING DONE**')
        printmd('All results saved in: ' + wdir +".")
        printmd('(If you need more detailed basin polygon, try to decrease stream lenght threshold (e.g. 1000).')

In [16]:
def transform_raster():
    
    global wdir
    global output_raster

    path=fdialog2.selected
    path=path.replace("\\", "/")
    filename=path 
    input_raster=gdal.Open(filename)
    output_name = filename.split("/")[-1][0:-4]
    output_raster=wdir+output_name+"_TRANSFORMED.tif"
    warp=gdal.Warp(output_raster,input_raster,dstSRS='EPSG:4326')
    warp=None # Closes the files
    
    with rasterio.open (output_raster) as r:
        printmd("Coordinate system of raster: ") 
        print(r.crs)

In [17]:
def HillShade():
    
    global output_raster_HS
    
    output_raster_HS = gdal.DEMProcessing(wdir+'DEM_HS.tif', 
                                          output_raster, 
                                          'hillshade', #slope, color-relief, TRI, TPI, roughness, hillshade
                                          options=[], 
                                          azimuth=80, 
                                          altitude=300) 
    
    output_raster_HS = wdir+'DEM_HS.tif'

In [18]:
def display_map():
    
    # zoom to layers
    map1.fit_bounds(map1.get_bounds(), padding=(20, 20))
    
    lc = folium.LayerControl(position='topleft')
    lc.add_to(map1)
            
    with out4:
        display(map1)

In [19]:
def results():
    
    global out4    
    
    out4 = widgets.Output(layout={'border': '2px solid black'})
    display(out4)       
    

In [20]:
def processing():
    
    global out2
    out2 = widgets.Output(layout={'border': '2px solid black', 'height':'1000px'})
    display(out2)
    
    with out2:
        display('will be processed here ..')

In [21]:
def msg():
    
    global out3
    
    out3 = widgets.Output(layout={'border': '2px solid black', 'height':'550px'})
    display(out3)
    
    with out3:
        display('...')

In [22]:
def preparation():
    
    global wbt
    global wdir
    global dem
    global delete_data
    global stream_t
    global bGen_only
    
    wbt = whitebox.WhiteboxTools()
    
    #working directory
    wdir = fdialog.selected
    wdir = wdir.replace("\\", "/")
    wbt.work_dir = os.path.dirname(wdir)    
    
    #input dem
    dem = fdialog2.selected
    dem = dem.replace("\\", "/")    
    
    #calculate only basins - not parameters
    bGen_only = check0.value
    
    #delete preliminary data
    delete_data = check1.value
    
    #stream length threshold for basin generation
    stream_t = txt1.value
    
    with out2:
        out2.clear_output()
        calculation()

In [23]:
def on_button1_clicked(b):  
    global map1
    
    with out3:
        out3.clear_output()
        printmd('computing ..')
    with out4:
        out4.clear_output()
        
    #Visualization in folium
    
    map1= folium.Map(control_scale=True).add_child(MeasureControl())

    folium.TileLayer(
        tiles="https://tiles.stadiamaps.com/tiles/stamen_terrain/{z}/{x}/{y}{r}.png",
        name='Stadia.StamenTerrain',
        minZoom = 0,
        maxZoom = 18,
        attr='&copy; <a href="https://www.stadiamaps.com/" target="_blank">Stadia Maps</a> &copy; <a href="https://www.stamen.com/" target="_blank">Stamen Design</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
    ).add_to(map1)


    folium.TileLayer(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        name='Esri.WorldStreetMap',
        attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
    ).add_to(map1)
    
    fs = plugins.Fullscreen(position='topright', 
                      title='Full Screen', 
                      title_cancel='Exit Full Screen', 
                      force_separate_button=False, 
                      )    
    fs.add_to(map1)
    
    formatter = "function(num) {return L.Util.formatNum(num, 5) + ' º ';};"
    
    mp = plugins.MousePosition(
                    position="topleft",
                    separator=" | ",
                    empty_string="Coordinates: __ | __",
                    lng_first=True,
                    num_digits=20,
                    prefix="Coordinates:",
                    lat_formatter=formatter,
                    lng_formatter=formatter,
                    )
    mp.add_to(map1)
        
    preparation()

def but1():
    
    global button1
    button1 = widgets.Button(description="OK", disabled=True)
    display(button1)    
    button1.on_click(on_button1_clicked)    

In [24]:
def inputsetting():
    
    out1 = widgets.Output(layout={'border': '2px solid black'})
    display(out1)   
    with out1:
        wkdchooser()
        demchooser()
        streamthresh()
        basin_gen()
        preliminary()
        but1()

# FLOW2024 - 1 - valley generation

In [25]:
inputsetting()

Output(layout=Layout(border='2px solid black'))

In [26]:
processing()

Output(layout=Layout(border='2px solid black', height='1000px'))

In [27]:
msg()

Output(layout=Layout(border='2px solid black', height='550px'))

In [28]:
results()

Output(layout=Layout(border='2px solid black'))